Aidan Coyle, afcoyle@uw.edu

2021/03/19

Roberts lab at SAFS


# 40_cbai4.0_indexcreation

This script produces a BLASTx annotation table by BLASTing C. bairdi transcriptome v4.0 against the Swiss-Prot/UniProt database

BLAST was done on Mox, so all commands are copy-pasted over, unless otherwise specified.

**Transcriptome**: Link and background info for cbai_transcriptomev4.0 available [here](https://robertslab.github.io/sams-notebook/2021/03/17/Transcriptome-Assembly-C.bairdi-Transcriptome-v4.0-Using-Trinity-on-Mox.html). 

Direct link to folder with data available [here](https://gannet.fish.washington.edu/Atumefaciens/20210317_cbai_trinity_RNAseq_transcriptome-v4.0/cbai_transcriptome_v4.0.fasta_trinity_out_dir/). 

Transcriptome md5sum is 6450d6f5650bfb5f910a5f42eef94913

In [ ]:
# Working from the login node of Mox, specifically gscratch/srlab/afcoyle
# Download cbai_transcriptomev4.0
[afcoyle@mox2 afcoyle]$ curl -o projects/cbai4.0_blastx/cbai_transcriptomev4.0.fasta \
-k https://gannet.fish.washington.edu/Atumefaciens/20210317_cbai_trinity_RNAseq_transcriptome-v4.0/cbai_transcriptome_v4.0.fasta_trinity_out_dir/cbai_transcriptome_v4.0.fasta

In [ ]:
# Verify checksum
[afcoyle@mox2 afcoyle]$ md5sum projects/cbai4.0_blastx/cbai_transcriptomev4.0.fasta | grep "6450d6f5650bfb5f910a5f42eef94913"

6450d6f5650bfb5f910a5f42eef94913  projects/cbai4.0_blastx/cbai_transcriptomev4.0.fasta

In [ ]:
# Make database for uniprot db
# Already completed for 30_hemat1.6_indexcreation so not redoing,
# command here for completeness. 
/gscratch/srlab/programs/ncbi-blast-2.8.1+/bin/makeblastdb \
-in /gscratch/srlab/blastdbs/uniprot_sprot_20200123/uniprot_sprot.fasta \
-dbtype prot \
-parse_seqids \
-out /gscratch/srlab/afcoyle/projects/cbai4.0_blastx/output/blastdbs/uniprot_blastdb

### Mox slurm script for BLASTx of hematodinium transcriptome v1.6

In [ ]:
#!/bin/bash
## Job Name
#SBATCH --job-name=afcoyle_cbai4.0blast
## Allocation Definition
#SBATCH --account=srlab
#SBATCH --partition=srlab
## Resources
## Nodes
#SBATCH --nodes=1
## Walltime (days-hours:minutes:seconds format)
#SBATCH --time=1-12:00:00
## Memory per node
#SBATCH --mem=120G
##turn on e-mail notification
#SBATCH --mail-type=ALL
#SBATCH --mail-user=afcoyle@uw.edu
## Specify the working directory for this job
#SBATCH --chdir=/gscratch/scrubbed/afcoyle


/gscratch/srlab/programs/ncbi-blast-2.8.1+/bin/blastx \
-task="blastx" \
-query /gscratch/srlab/afcoyle/projects/cbai4.0_blastx/cbai_transcriptomev4.0.fasta \
-db /gscratch/srlab/afcoyle/projects/cbai4.0_blastx/output/blastdbs/uniprot_blastdb \
-out /gscratch/srlab/afcoyle/projects/cbai4.0_blastx/output/cbai4.0_blastxres.tab \
-evalue 1E-05 \
-num_threads 40 \
-max_target_seqs 1 \
-outfmt 6

In [ ]:
# Send job to executive node of Mox
[afcoyle@mox1 afcoyle]$ sbatch jobs/20210319_cbai4.0_blastx.sh
Submitted batch job 1760093

In [ ]:
# Transfer BLAST output from Mox to Gannet
# These commands performed from a folder in Gannetc
# Gannet folder: /volume2/web/nerka/mox_transfers/scrubbed/
rsync -avz --progress \
afcoyle@mox.hyak.uw.edu:/gscratch/srlab/afcoyle/projects/cbai4.0_blastx/output/cbai4.0_blastxres.tab \
hemat_proj/

In [ ]:
# Check our md5sums to ensure correct transfer. Here's what they should be:
# To obtain, run md5sum cbai4.0_blastxres.tab on both Mox and Gannet

# e9c40aae5f5897a1cb832448dcb96f85  cbai4.0_blastxres.tab

In [ ]:
# Transfer both BLAST outputs to local machine
# Ran command from local machine
# Using absolute path, as relative path fails
rsync -chavzP --stats \
afcoyle@gannet.fish.washington.edu:/volume2/web/nerka/mox_transfers/scrubbed/hemat_proj/cbai4.0_blastxres.tab \
/mnt/c/Users/acoyl/Documents/GitHub/hemat_bairdi_transcriptome/output/BLASTs/uniprot_swissprot/

In [ ]:
# Verify checksums still match by running md5sum ../output/BLASTs/uniprot_swissprot/cbai4.0_blastxres.tab on local machine

# e9c40aae5f5897a1cb832448dcb96f85  cbai4.0_blastxres.tab